In this section, we will implement the Google PageRank algorithm using two different approaches. First, we will view it as a Markov process, simulating the random walk behavior of a user moving between webpages. Then, we will take a more deterministic approach by solving the problem using matrix factorization, which involves setting up and solving a system of linear equations.

We will also explain why both methods ultimately produce the same ranking after convergence.

Before diving into the PageRank computation, we will generate a graph represented by an adjacency matrix for a directed graph. In this graph, each outgoing link from a node represents a hyperlink from the current webpage to another webpage, and each incoming edge to a node represents a hyperlink from another webpage to the current one.

In [1]:
''' import libraries '''

import networkx as nx
import numpy as np
import plotly.graph_objects as go

In [46]:
np.random.seed(42)

In [47]:
''' generate adjacency matrix '''

def generateADJ(n = 10, density = 0.3):
    """
    Generate an n×n adjacency matrix where the
    ratio of ones to n^2 is approximately equal
    to the specified density parameter.
    """
    adj_matrix = np.zeros((n, n), dtype=int)
    num_edges = int(density * n * (n - 1) / 2)
    edges = set()
    while len(edges) < num_edges:
        u, v = np.random.randint(0, n - 1), np.random.randint(0, n - 1)
        if u != v:
            edge = tuple(sorted((u, v)))
            edges.add(edge)
    for u, v in edges:
        adj_matrix[u, v] = 1
        adj_matrix[v, u] = 1

    return adj_matrix

adj = generateADJ()
adj

array([[0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 1, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 1, 0, 1, 0],
       [0, 1, 1, 1, 0, 0, 0, 1, 1, 0],
       [1, 0, 1, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 1, 1, 0, 0, 0],
       [1, 0, 0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [48]:
''' construct the graph using nx '''

def create_graph(adj):
    """
    Constructs a directed graph from the given adjacency matrix.
    """
    rows, cols = np.where(adj == 1)
    G = nx.DiGraph()
    for u, v in zip(rows, cols):
        G.add_edge(u, v)
    G.add_nodes_from(range(adj.shape[0]))
    return G

graph = create_graph(adj)
graph

In [49]:
''' visualize your generated graph '''

"""
Just run this cell.
Whenever you want to visualize the graph based on
your current values, activate the test mine flag.
"""


def visualize_graph(G, testMine = False , myProbs = []):
    pos = nx.spring_layout(G, seed=42)

    edge_x = []
    edge_y = []
    arrow_x = []
    arrow_y = []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)

        arrow_x.append((x0 + x1) / 2)
        arrow_y.append((y0 + y1) / 2)

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=1, color='black'),
        hoverinfo='none',
        mode='lines'
    )

    arrow_trace = go.Scatter(
        x=arrow_x, y=arrow_y,
        mode='markers',
        marker=dict(
            size=8,
            color='black',
            symbol='triangle-up',
            angle=-90
        ),
        hoverinfo='none'
    )

    node_x = []
    node_y = []
    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers',
        hoverinfo='text',
        marker=dict(
            showscale=True,
            colorscale='YlGnBu',
            size=20,
            color=[],
            colorbar=dict(
                thickness=15,
                title='PageRank Value',
                xanchor='left',
                titleside='right'
            ),
            line_width=2))

    node_color = []
    node_text = []
    pagerank_values = nx.pagerank(G)
    for node in G.nodes():
        if testMine:
            node_color.append(myProbs[node])
            node_text.append(f'Page {node} - PageRank: {myProbs[node]:.3f}')
        else:
            node_color.append(pagerank_values[node])
            node_text.append(f'Page {node} - PageRank: {pagerank_values[node]:.3f}')

    node_trace.marker.color = node_color
    node_trace.text = node_text

    fig = go.Figure(data=[edge_trace, arrow_trace,  node_trace],
                    layout=go.Layout(
                        title='Webpage Linkage Graph (PageRank Visualization)',
                        titlefont_size=16,
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=0, l=0, r=0, t=40),
                        annotations=[dict(
                            text="Directed graph representing webpage linkages",
                            showarrow=False,
                            xref="paper", yref="paper",
                            x=0.005, y=-0.002)],
                        xaxis=dict(showgrid=False, zeroline=False),
                        yaxis=dict(showgrid=False, zeroline=False)))

    fig.show()
    return pagerank_values

goalProbs = visualize_graph(graph)
print(f'Actual pagerank values : {goalProbs}')
myProbs = {key : 1/len(goalProbs) for key in goalProbs}
visualize_graph(graph, testMine=True , myProbs=myProbs)
print(f'Values I found : {myProbs}')


Actual pagerank values : {0: 0.08075637215390871, 5: 0.11375701670615261, 8: 0.11340487050803304, 1: 0.04759071317705914, 4: 0.18351200783878102, 2: 0.11125229689907729, 6: 0.11092869368937, 3: 0.11115228950559007, 7: 0.11125229689907729, 9: 0.016393442622950824}


Values I found : {0: 0.1, 5: 0.1, 8: 0.1, 1: 0.1, 4: 0.1, 2: 0.1, 6: 0.1, 3: 0.1, 7: 0.1, 9: 0.1}


Repeat the 3 cells above as many times as necessary until you reach a graph you are happy with. Keep in mind, as you have seen, the plot is interactive, allowing you to see the names of the nodes and their assigned values. Additionally, there are two extra parameters passed to the function:

1. The first parameter is for testing mode, which lets you view the ranking based on your current state values. If not in testing mode, the default is the true ranking value.
   
2. The second parameter is the dictionary you have prepared for your test.

From here, we will explore a Markovian process to determine the stationary ranking we can reach by setting the convergence threshold. We start from uniformly distributed probabilities.


In the transposed version of the adjacency matrix, column *i* indicates to which other nodes we can proceed. By normalizing the columns so that each sums up to one, we can observe the probability of transitioning from node *i* to node *j*. This means that, to determine the probability of moving from node *i* to node *j*, we look at column *i* and row *j*.

When this matrix is multiplied by the current distribution of probabilities (the probability of being at each node, stored in `myProbs`), we get new probabilities. These represent the updated distribution after one iteration, according to the equation:

$
P_{t+1} = T \cdot P_t
$

Here, we implement this process over a certain number of iterations and plot the difference between the current probability vector and the previous one at each iteration. This allows us to track the convergence and understand when the distribution stabilizes.


In [50]:
''' Simulate the random walk '''

def find_stationary_values(transitionMatrix, currentValues, iterations = 150, threshold = 1e-15):
    """
    Parameters:

    - transitionMatrix : adj matrix which you need to normalize.
    - currentValues : initial probabilities which we initialized with a unifirm distribution.
    - iterations : The maximum number of iterations to perform the random walk.
    - treshold: The threshold for the difference ( you can use L1 norm ) between consecutive iterations.
                The simulation stops when the difference falls below this value.
    """
    column_sums = transitionMatrix.sum(axis=0)
    dangling_nodes = column_sums == 0
    transitionMatrix[:, dangling_nodes] = 1
    column_sums = transitionMatrix.sum(axis=0)
    normalizedTransitions = transitionMatrix / column_sums[np.newaxis, :]
    myProbs = np.array(currentValues)
    for i in range(iterations):
          nextProbs = normalizedTransitions @ myProbs
          diff = np.linalg.norm(nextProbs - myProbs, ord=1)
          print(f'iteration {i} | diff {diff}')
          if diff < threshold:
              print(f"Converged after {i + 1} iterations.")
              break
          myProbs = nextProbs

    visualize_graph(graph, testMine=True, myProbs=myProbs)
    visualize_graph(graph)

    return normalizedTransitions



normalizedTransitions = find_stationary_values(adj, list(myProbs.values()))

iteration 0 | diff 0.3866666666666666
iteration 1 | diff 0.24422222222222228
iteration 2 | diff 0.20700740740740747
iteration 3 | diff 0.18061506172839512
iteration 4 | diff 0.16194376954732514
iteration 5 | diff 0.1477317028806584
iteration 6 | diff 0.13423156389574759
iteration 7 | diff 0.12200914327480573
iteration 8 | diff 0.11072133469093144
iteration 9 | diff 0.10044787971705449
iteration 10 | diff 0.09106755326799201
iteration 11 | diff 0.08254236775583222
iteration 12 | diff 0.07479347925885108
iteration 13 | diff 0.06776175709508557
iteration 14 | diff 0.061382756406146266
iteration 15 | diff 0.05559966273767423
iteration 16 | diff 0.05035808042374788
iteration 17 | diff 0.04560865798304633
iteration 18 | diff 0.041305810920883944
iteration 19 | diff 0.037408066841843654
iteration 20 | diff 0.03387756772939922
iteration 21 | diff 0.030679917691306187
iteration 22 | diff 0.02778385783457575
iteration 23 | diff 0.02516102695484499
iteration 24 | diff 0.02278569875591508
iteratio

You might encounter some dangling nodes in the graph, which are nodes with no outgoing edges. When the random walker reaches such a node, no other node is reachable from it, potentially causing `NaN` values in the normalized transition matrix. To handle this, you can assume that the random walker can move to any other node with equal probability.

In the context of the transition matrix, this means that you should adjust the matrix to account for these dangling nodes. Specifically, for dangling nodes, replace the corresponding columns with a uniform distribution, ensuring that each node has an equal chance of being the next node.


## Using decomposition

Now that we have achieved a ranking that closely matches the actual values, we can proceed with a new approach.

When we reach the stable distribution, the transition matrix $ T $ will no longer affect the current state. In other words, we are looking for a vector $ p $ such that:

$ p_t = T p_t $

Solving for $ p $ is equivalent to finding the eigenvector corresponding to the eigenvalue 1 of $ T $. To do this, we will use QR decomposition to solve the equation.

### Tasks

1. **Implement an Efficient System of Equation Solver**:
   - Use QR decomposition to solve $ (T - I)p = 0 $, where $ I $ is the identity matrix.
   - - To find the null space of $ T - I $, we use the Gram-Schmidt process on $ (T - I)^T $ to obtain an orthonormal basis for the row space of $ T - I $. When examining vectors corresponding to larger diagonal values of $ R $, we can determine that these vectors do contribute significantly to the basis of the row space, while the remaining vectors form the null space. This method provides an approximate solution that is close to the exact answer. However, be aware that dividing by the norm in the Gram-Schmidt process can lead to numerical instability, so increasing the precision of calculations is advisable.

2. **Compare Results**:
   - Compare the computed $ p $ with the actual values to evaluate the accuracy.

For a deeper understanding of the PageRank algorithm, you can watch this video: [PageRank Algorithm](https://www.youtube.com/watch?v=TU0ankRcHmo).


In [58]:
''' Implement QR '''

def gram_schmidt_qr(A):
    """
    Perform QR decomposition using the Gram-Schmidt process.

    Parameters:
    - A (numpy.ndarray): A 2D matrix (m x n) to be decomposed, where m is the number of rows
                          and n is the number of columns. The matrix should be of full column rank.

    Returns:
    - Q (numpy.ndarray): An m x n orthogonal matrix.
    - R (numpy.ndarray): An n x n upper triangular matrix.
    """
    m, n = A.shape
    Q = np.zeros((m, n))
    R = np.zeros((n, n))

    for j in range(n):
        v = A[:, j]
        for i in range(j):
            R[i, j] = np.dot(Q[:, i].T, v)
            v = v - R[i, j] * Q[:, i]
        R[j, j] = np.linalg.norm(v)
        Q[:, j] = v / R[j, j]
    return Q, R

def compute_null_space_qr(A, tol=1e-15):

    """
    After performing the Gram-Schmidt process,
    sort the columns of the Q matrix based on
    their corresponding values in the R matrix.
    Select the columns with smaller values in R
    (below a given threshold) as the null space vectors.
    Then, return the largest vector from the null space,
    normalized to form a valid probability distribution.
    """

    Q, R = gram_schmidt_qr(A.T)
    null_space_vectors = Q[:, np.abs(np.diag(R)) < tol]
    if null_space_vectors.shape[1] > 0:
        largest_vector = null_space_vectors[:, np.argmax(np.linalg.norm(null_space_vectors, axis=0))]
        myProbs = np.exp(largest_vector) / np.sum(np.exp(largest_vector))

    visualize_graph(graph, testMine=True, myProbs=myProbs)
    visualize_graph(graph)

    return myProbs



null_space = compute_null_space_qr(normalizedTransitions - np.eye(adj.shape[0]))